In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from datasets import load_dataset

ds = load_dataset("bigcode/starcoderdata", data_dir="java", split="train")

Resolving data files:   0%|          | 0/87 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/181 [00:00<?, ?it/s]

In [2]:
streamed_ds = load_dataset("bigcode/starcoderdata", data_dir="java", split="train", streaming=True)

Resolving data files:   0%|          | 0/87 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder")

In [4]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("pad_token set to eos_token")

pad_token set to eos_token


In [5]:
from transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=1024,
    n_embd=512,
    n_layer=24,
    n_head=8
)

model = GPT2LMHeadModel(config)

In [6]:
print(model.num_parameters())

101348352


In [7]:
example = next(iter(streamed_ds))
print("Dataset keys:", example.keys())
print("Example:", example)

Dataset keys: dict_keys(['max_stars_repo_path', 'max_stars_repo_name', 'max_stars_count', 'id', 'content'])
Example: {'max_stars_repo_path': 'dashbuilder-shared/dashbuilder-dataset-api/src/main/java/org/dashbuilder/dataset/DataSetConstraints.java', 'max_stars_repo_name': 'shanika04/dashbuilder', 'max_stars_count': 0, 'id': '0', 'content': '/*\n * Copyright 2014 Red Hat, Inc. and/or its affiliates.\n *\n * Licensed under the Apache License, Version 2.0 (the "License");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *       http://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an "AS IS" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\npackage org.dashbuild

In [8]:
def tokenize_function(examples):
    return tokenizer(
        examples["content"],
        truncation=True,
        max_length=1024,
        padding="max_length"
    )

tokenized_dataset = streamed_ds.map(tokenize_function, batched=True, remove_columns=["content"])

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=1,
    fp16=True,
    logging_steps=100,
    save_steps=1000,
    learning_rate=5e-5,
    gradient_accumulation_steps=8,
    max_steps=2508971, # calculated from the next cell
    report_to="none"
)

In [12]:
# Calculate the number of steps for 1 epoch
total_samples = len(ds)
effective_batch_size = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
steps_per_epoch = total_samples // effective_batch_size

print(f"Total samples: {total_samples}")
print(f"Effective batch size: {effective_batch_size}")
print(f"Steps per epoch: {steps_per_epoch}")

Total samples: 20071773
Effective batch size: 8
Steps per epoch: 2508971


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()
trainer.save_model("./model")
trainer.save_pretrained("./model")

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,8.121200


KeyboardInterrupt: 